In [1]:
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Single value for mean and std for grayscale
])

fashion_dataset=torchvision.datasets.FashionMNIST(root='./data',train=True,download=True,transform=transform)
trainset=torch.utils.data.DataLoader(fashion_dataset,batch_size=4,shuffle=True)

fashion_dataset_test=torchvision.datasets.FashionMNIST(root='./data',train=False,download=True,transform=transform)
testset=torch.utils.data.DataLoader(fashion_dataset_test,batch_size=4,shuffle=True)


In [3]:
class DeepNN(nn.Module):
    def __init__(self,hidden_size1=500,hiddensize2=1000):
        super(DeepNN,self).__init__()
        self.linear1=nn.Linear(784,hidden_size1)
        self.linear2=nn.Linear(hidden_size1,hiddensize2)
        self.linear3=nn.Linear(hiddensize2,10)
        self.relu=nn.LeakyReLU()

    def forward(self,img):
        x=img.view(-1,784) 
        x=self.relu(self.linear1(x))
        x=self.relu(self.linear2(x))
        x=self.linear3(x)
        return x
    
net=DeepNN()

In [5]:
def train(trainset,net,epochs=5):
    cross=nn.CrossEntropyLoss()
    optim=torch.optim.Adam(net.parameters(),lr=0.001)
    totalit=0

    for epoch in range(epochs):
        net.train()
        runningloss=0.0
        for i,(input,labels) in   :
            optim.zero_grad()
            outputs=net(input)
            loss=cross(outputs,labels)
            loss.backward()
            optim.step()

            runningloss+=loss.item()

        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {runningloss / len(trainset):.4f}") 
    print('Finished Training')



        
train(trainset,net,epochs=1)

Epoch [1/1], Loss: 0.4378
Finished Training


In [7]:
from sklearn.metrics import classification_report, accuracy_score
def test():
    y_pred=[]
    y_true=[]

    with torch.no_grad():
        net.eval()
        for data in testset:
            images,labels=data
            x,y=data
          
            x=x.view(-1,784)
            outputs=net(x)   
            predictions=torch.argmax(outputs,dim=1)
            y_true.extend(y.numpy())
            y_pred.extend(predictions.numpy())

    print(f"Classification Report")
    print(classification_report(y_true, y_pred, target_names=fashion_dataset.classes))
    print(f"Overall Accuracy: {accuracy_score(y_true, y_pred):.3f}")

test()


    
    

Classification Report
              precision    recall  f1-score   support

 T-shirt/top       0.79      0.74      0.77      1000
     Trouser       0.98      0.94      0.96      1000
    Pullover       0.88      0.55      0.68      1000
       Dress       0.71      0.94      0.81      1000
        Coat       0.72      0.80      0.76      1000
      Sandal       0.93      0.88      0.91      1000
       Shirt       0.63      0.57      0.60      1000
     Sneaker       0.94      0.80      0.86      1000
         Bag       0.75      0.97      0.85      1000
  Ankle boot       0.90      0.96      0.93      1000

    accuracy                           0.81     10000
   macro avg       0.82      0.81      0.81     10000
weighted avg       0.82      0.81      0.81     10000

Overall Accuracy: 0.815


In [10]:
from torchinfo import summary
summary(net, input_size=(4, 1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
DeepNN                                   [4, 10]                   --
├─Linear: 1-1                            [4, 500]                  392,500
├─LeakyReLU: 1-2                         [4, 500]                  --
├─Linear: 1-3                            [4, 1000]                 501,000
├─LeakyReLU: 1-4                         [4, 1000]                 --
├─Linear: 1-5                            [4, 10]                   10,010
Total params: 903,510
Trainable params: 903,510
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 3.61
Input size (MB): 0.01
Forward/backward pass size (MB): 0.05
Params size (MB): 3.61
Estimated Total Size (MB): 3.67

# Lets say rank is 8

In [ ]:
class LoRA(nn.Module):
    def __init__(self,original_layer,rank=8):
        super().__init__()
        self.rank=rank
        self.original_layer=original_layer

        in_features=self.original_layer.in_features
        out_feature=self.original_layer.out_features
        self.A=nn.Parameter(torch.rand(in_features,rank))
        self.B=nn.Parameter()
        